In [42]:
from collections import defaultdict, Counter
import json
from matplotlib import pyplot as plt
import numpy as np
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import os
from datasets import DatasetDict, load_dataset, Dataset


path_to_data_folder="/Users/matteom/shared-folder/nlpt_group/project/data"

In [51]:
# Read in abstracts
df_part1 = pd.read_csv(os.path.join(path_to_data_folder,"processed_data_part1.csv"))
df_part2 = pd.read_csv(os.path.join(path_to_data_folder,"processed_data_part2.csv"))
df=pd.concat([df_part1,df_part2])
abstracts = df[['Abstract']] #.tolist()
print(abstracts[:3])

                                            Abstract
0  SUMMARY Several lines of evidence support the ...
1  Acute inflammation is a severe medical conditi...
2  Human brain connectivity can be studied using ...


In [53]:
abstracts_dataset = Dataset.from_pandas(abstracts)
abstracts_dataset.set_format("torch")
abstracts_dataset

Dataset({
    features: ['Abstract', '__index_level_0__'],
    num_rows: 58850
})

In [6]:
# Load BioBERT
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
model = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.2")

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [56]:
split=int(np.floor(0.7*len(abstracts)))

# Take 70% of abstracts for train and 30% for validation
abstracts_dataset_dict = DatasetDict(
    train=abstracts_dataset.shuffle(seed=19).select(range(split)),
    val=abstracts_dataset.shuffle(seed=19).select(range(split, len(abstracts))),
)
abstracts_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['Abstract', '__index_level_0__'],
        num_rows: 41195
    })
    val: Dataset({
        features: ['Abstract', '__index_level_0__'],
        num_rows: 17655
    })
})

In [36]:
tokenized_abstracts = abstracts_dataset.map(
    lambda abstract: tokenizer(abstract, return_tensors="pt", truncation=True, padding=True, max_length=512),
    batched=True,  #so the function is applied to multiple elements of our dataset at once, and not on each element separately,
    batch_size=16,
    num_proc=2# number of workers
)
tokenized_abstracts

TypeError: Values in `DatasetDict` should be of type `Dataset` but got type '<class 'list'>'